In [2]:
import pandas as pd

In [3]:
%config Completer.use_jedi = False

In [5]:
from ast import literal_eval

In [16]:
from gensim import models

In [12]:
from nltk.corpus import stopwords

In [53]:
from gensim.utils import simple_preprocess

In [21]:
from nltk import sent_tokenize

In [10]:
df = pd.read_csv(r"D:\data\openparliament\statements_nospeaker_en.csv",index_col="Unnamed: 0")

C:\Users\alecr\Anaconda3\envs\gensim_env\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (20,23,28,29,30,31,32,33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
df.head()

,id,politician_id,riding_id,party_id,end_date,start_date,id.1,document_id,time,h1_en,...,slug,urlcache,h1_fr,h2_fr,h3_fr,who_fr,who_context_fr,wordcount_en,tokenized_content,slug_length
0,2611,3465,35066,4,2005-11-29,1997-09-22,232373,1878,2001-05-03 13:50:00-04,Government Orders,...,marlene-catterall-1,/debates/2001/5/3/marlene-catterall-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'on', 'a'...",3
1,3210,3465,70224,4,1997-04-27,1994-01-17,232373,1878,2001-05-03 13:50:00-04,Government Orders,...,marlene-catterall-1,/debates/2001/5/3/marlene-catterall-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'on', 'a'...",3
2,4305,173,70358,1,\N,2015-10-19,645329,388,2008-02-14 13:15:00-05,Routine Proceedings,...,tom-lukiwski-7,/debates/2008/2/14/tom-lukiwski-7/,NaN,NaN,NaN,NaN,NaN,\N,"['finally', ',', 'mr.', 'speaker', ',', 'i', '...",3
3,1534,173,47007,1,2015-10-19,2004-10-04,645329,388,2008-02-14 13:15:00-05,Routine Proceedings,...,tom-lukiwski-7,/debates/2008/2/14/tom-lukiwski-7/,NaN,NaN,NaN,NaN,NaN,\N,"['finally', ',', 'mr.', 'speaker', ',', 'i', '...",3
4,1541,43,47014,1,2015-10-19,2004-10-04,313253,1621,2004-02-06 10:05:00-05,NaN,...,garry-breitkreuz-1,/debates/2004/2/6/garry-breitkreuz-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'today', ...",3


In [13]:
stop_words = set(stopwords.words('english'))

In [14]:
df['tokens'] = df.tokenized_content.apply(literal_eval)

In [15]:
%%time
df['tokens'] = df.tokens.apply(lambda x: [w for w in x if not w in stop_words])

Wall time: 30.1 s


steps:
    sent_tokenize
    simple_preprocess and remove punctuation
    remove stopwords
    then gensim

In [22]:
from bs4 import BeautifulSoup

In [30]:
%%time
df['text_en'] = df.content_en.apply(lambda x: BeautifulSoup(x).text)

Wall time: 3min 18s


In [25]:
#save souped df

In [31]:
df.columns

Index(['id', 'politician_id', 'riding_id', 'party_id', 'end_date',
       'start_date', 'id.1', 'document_id', 'time', 'h1_en', 'h2_en',
       'member_id', 'who_en', 'content_en', 'sequence_en', 'wordcount',
       'politician_id.1', 'procedural', 'h3_en', 'who_hocid', 'content_fr',
       'statement_type', 'written_question', 'source_id', 'who_context_en',
       'slug', 'urlcache', 'h1_fr', 'h2_fr', 'h3_fr', 'who_fr',
       'who_context_fr', 'wordcount_en', 'tokenized_content', 'slug_length',
       'tokens', 'text_en'],
      dtype='object')

In [ ]:
df.drop(['urlcache', 'h1_fr', 'h2_fr', 'h3_fr', 'who_fr',
       'who_context_fr', 'wordcount_en', 'tokenized_content','tokens'],axis=1)

In [33]:
df.to_csv(r"D:\data\openparliament\text_en.csv",index = False)

In [64]:
%%time
#apply sent_tokenize
df['sentences'] = df.text_en.apply(sent_tokenize)

Wall time: 3min 50s


In [27]:
#apply simple_preprocess to sentence column

In [65]:
%%time
sentences = df.sentences.apply(lambda x: [simple_preprocess(s) for s in x])

Wall time: 3min 47s


In [28]:
#save column

In [66]:
sentences.to_csv(r"D:\data\openparliament\sentences_tokenized.csv")

In [29]:
#gensim

In [67]:
type(sentences)

pandas.core.series.Series

In [68]:
sentences.shape

(861398,)

In [69]:
sentences.iloc[0]

[['mr', 'speaker', 'rise', 'on', 'point', 'of', 'order'],
 ['believe',
  'you',
  'would',
  'find',
  'unanimous',
  'consent',
  'following',
  'consultation',
  'among',
  'all',
  'parties',
  'to',
  'further',
  'defer',
  'the',
  'vote',
  'just',
  'deferred',
  'until',
  'monday',
  'to',
  'next',
  'tuesday',
  'at',
  'the',
  'end',
  'of',
  'government',
  'orders']]